In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/wikihow-summarization/wikihowAll.csv', delimiter = ',')
df.head()

In [ ]:
import numpy as np  
import pandas as pd 

import re
import nltk

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional

#embedding : we use an embedding layer to compress the input feature space into a smaller one.
#The Embedding layer is used to create word vectors for incoming words.
#A dense layer is just a regular layer of neurons in a neural network.


from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

#import warnings
#pd.set_option("display.max_colwidth", 200)
#warnings.filterwarnings("ignore")

In [ ]:
df.shape

In [ ]:
DF = df.dropna()

In [ ]:
import nltk
nltk.download('stopwords')
STOP_WORDS = set(stopwords.words('english')) 

In [ ]:
DF.shape

In [ ]:
DF.head()

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
DF.shape

In [ ]:

def CLEAN_TEXT(text):

    TEXT = text.lower()
    TEXT = re.sub(r'\([^)]*\)', '', TEXT)
    TEXT = re.sub('"','', TEXT)

    TEXT = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in TEXT.split(" ")])    

    TEXT = re.sub(r"'s\b","", TEXT)
    TEXT = re.sub("[^a-zA-Z]", " ", TEXT) 
    tokens = [w for w in TEXT.split() if not w in STOP_WORDS]
    #tokens = TEXT.split()

    long_words = []
    
    for i in tokens:
        if len(i) >= 3:                
            long_words.append(i)   
    return (" ".join(long_words)).strip()
    
    #return TEXT

cleaned_text = []

for t in DF['text']:
    cleaned_text.append(CLEAN_TEXT(t))

In [ ]:
def CLEAN_SUMMARY(text):

    TEXT = re.sub('"','', text)

    TEXT = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in TEXT.split(" ")])  

    TEXT = re.sub(r"'s\b","",TEXT)
    TEXT = re.sub("[^a-zA-Z]", " ", TEXT)
    TEXT = TEXT.lower()
    #return TEXT
    tokens = TEXT.split()

    corpus = ''

    for i in tokens:
        if len(i)>1:                                 
            corpus = corpus + i + ' '  
    return corpus

cleaned_summary = []

for t in DF['headline']:
    cleaned_summary.append(CLEAN_SUMMARY(t))

DF['cleaned_text'] = cleaned_text
DF['cleaned_summary'] = cleaned_summary
DF['cleaned_summary'].replace('', np.nan, inplace = True)

DF.dropna(axis = 0, inplace = True)

In [ ]:
for i in range(5):
    print("TEXT : ",DF['cleaned_text'][i])
    print("SUMMARY : ",DF['cleaned_summary'][i])
    print("\n")

In [ ]:
text_word_count = []
summary_word_count = []

# populate the lists with sentence lengths
for i in DF['cleaned_text']:
      text_word_count.append(len(i.split()))

for i in DF['cleaned_summary']:
      summary_word_count.append(len(i.split()))

length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})

In [ ]:
length_df.hist(bins = 40, figsize = (15, 5))
plt.show()

In [ ]:
max_text_len = 70
max_summary_len = 20

In [ ]:
DF['cleaned_summary'] = DF['cleaned_summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [ ]:
DF['cleaned_summary']

In [ ]:
from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(np.array(DF['cleaned_text']), np.array(DF['cleaned_summary']),
                                            test_size = 0.1,
                                            random_state = 0,
                                            shuffle = True)

In [ ]:
import pickle

X_TRAIN = open('x_tr_metrics.pickle', 'wb')
pickle.dump(x_tr, X_TRAIN)
X_TRAIN.close()

Y_TRAIN = open('y_tr_metrics.pickle', 'wb')
pickle.dump(y_tr, Y_TRAIN)
Y_TRAIN.close()

Y_VAL = open('y_val_metrics.pickle', 'wb')
pickle.dump(y_val, Y_VAL)
Y_VAL.close()

X_VAL = open('x_val_metrics.pickle', 'wb')
pickle.dump(x_val, X_VAL)
X_VAL.close()

In [ ]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [ ]:
thresh = 4

cnt = 0
tot_cnt = 0
freq = 0
tot_freq = 0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt = tot_cnt + 1
    tot_freq = tot_freq + value
    if(value < thresh):
        cnt = cnt + 1
        freq = freq + value
    
print("% of rare words in vocabulary:", (cnt/tot_cnt) * 100)
print("Total Coverage of rare words:", (freq/tot_freq) * 100)

In [ ]:
x_tokenizer = Tokenizer(num_words = tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences (i.e one-hot encodeing all the words)
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen = max_text_len, padding = 'post') #préserver la dimension des features
x_val   =   pad_sequences(x_val_seq, maxlen = max_text_len, padding = 'post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

In [ ]:
#prepare a tokenizer for reviews on training data

y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [ ]:
cnt = 0
tot_cnt = 0
freq = 0
tot_freq = 0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt = tot_cnt + 1
    tot_freq = tot_freq + value
    
    if(value < thresh):
        cnt = cnt + 1
        freq = freq + value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt) * 100)
print("Total Coverage of rare words:",(freq/tot_freq) * 100)

In [ ]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words = tot_cnt - cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen = max_summary_len, padding = 'post')
y_val   =   pad_sequences(y_val_seq, maxlen = max_summary_len, padding = 'post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words + 1
print("Size of vocabulary in Y = {}".format(y_voc))

In [ ]:
ind = []
for i in range(len(y_tr)):
    cnt = 0
    for j in y_tr[i]:
        if j != 0:
            cnt = cnt + 1
    if(cnt == 2):
        ind.append(i)

y_tr = np.delete(y_tr, ind, axis = 0)
x_tr = np.delete(x_tr, ind, axis = 0)

In [ ]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j != 0:
            cnt = cnt + 1
    if(cnt==2):
        ind.append(i)

y_val = np.delete(y_val, ind, axis = 0)
x_val = np.delete(x_val, ind, axis = 0)

In [ ]:
latent_dim = 300
embedding_dim = 200

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim, trainable = True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences = True,return_state = True, dropout = 0.4, recurrent_dropout = 0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3 = LSTM(latent_dim, return_state = True, return_sequences = True, dropout = 0.4, recurrent_dropout = 0.4)
encoder_output3, state_h3, state_c3 = encoder_lstm3(encoder_output2)

#encoder lstm 4
#encoder_lstm4 = LSTM(latent_dim, return_state = True, return_sequences = True, dropout = 0.4, recurrent_dropout = 0.4)

#encoder_output4, state_h4, state_c4 = encoder_lstm4(encoder_output3)

encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output3)
#encoder_outputs, state_h, state_c= encoder_lstm2(encoder_output1)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape = (None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim, trainable = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state = True, dropout = 0.4, recurrent_dropout = 0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


model.summary()

In [ ]:
#from keras.callbacks import ModelCheckPoint

In [ ]:
model.compile(optimizer = 'rmsprop',
              loss = 'sparse_categorical_crossentropy')

es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   verbose = 1,
                   patience = 2)

mc = ModelCheckpoint('ALL_ATTENTION_best_model_with_no_stopwords_METRICS.h5',
                    monitor = 'val_loss',
                    mode = 'min',
                    verbose = 1,
                    save_best_only = True)

history = model.fit([x_tr, y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:,1:],
                    epochs = 50,
                    callbacks = [es, mc],
                    batch_size = 128,
                    validation_data = ([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:,1:]))

In [ ]:
#haha

In [ ]:
#de 9 à 20

#from keras.models import load_model
#from tensorflow.keras.models import load_model

#new_model = tf.keras.models.load_model('../input/models/ALL_best_model.h5')

#new_model.summary()

#new_model.compile(optimizer = "rmsprop", loss = 'sparse_categorical_crossentropy')
#es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
#checkpoint = ModelCheckpoint('ALL_best_model_7_ 50_epochs.h5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min', period = 1)
#history = new_model.fit([x_tr, y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:,1:],
#                    epochs = 50,
#                    callbacks = [es, checkpoint],
#                    batch_size = 128,
#                    validation_data = ([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:,1:]))

In [ ]:
from matplotlib import pyplot
pyplot.figure(figsize = (11,8))
pyplot.plot(history.history['loss'], label = 'train', color = 'black')
pyplot.plot(history.history['val_loss'], label = 'test', color = 'purple')
pyplot.legend()
pyplot.show()
pyplot.savefig('plot.jpeg')

In [ ]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

# Encode the input sequence to get the feature vector

encoder_model = Model(inputs = encoder_inputs, outputs = [encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_hidden_state_input = Input(shape = (max_text_len, latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = [decoder_state_input_h, decoder_state_input_c])

In [ ]:
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
import pickle
#from tf.keras.model import save_model
#from tf.keras.models import load_model

#model.save_model('standard_model.h5')
#m = open('Attention_model.pickle', 'wb')
#pickle.dump(model, m)
#m.close()

encoder_model.save('encoder_metrics.h5')
decoder_model.save('decoder_metrics.h5')

x = open('x_tokenizer_metrics.pickle', 'wb')
pickle.dump(x_tokenizer, x)
x.close()

y = open('y_tokenizer_metrics.pickle', 'wb')
pickle.dump(y_tokenizer, y)
y.close()

In [ ]:
#save that model, honey boo~
from keras.models import save_model

In [ ]:
#model = save_model('../input/output/MODEL.h5')
model.save_weights('ATTENTION_MODEL_WEIGHTS.h5')

In [ ]:
#BROUILLON

#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('my_model.h5')

#model = load_model('Attention_model.h5')
#model.save('Attention_model.h5')

#encoder_model.save('Attention_encoder.h5')

#decoder_model.save('Attention_decoder.h5')

#x = open('Attention_x_tokenizer.pickle', 'wb')
#pickle.dump(x_tokenizer, x)
#x.close()

#y = open('Attention_y_tokenizer.pickle', 'wb')
#pickle.dump(y_tokenizer, y)
#y.close()

In [ ]:
#m = open('Attention_model.pickle', 'wb')#
#pickle.dump(model, m)
#m.close()

#encoder = open('Attention_encoder_model.pickle', 'wb')
#pickle.dump(encoder_model, encoder)
#encoder.close()

#decoder = open('Attention_decoder_model.pickle', 'wb')
#pickle.dump(decoder_model, decoder)
#decoder.close()

#x = open('Attention_x_tokenizer.pickle', 'wb')
#pickle.dump(x_tokenizer, x)
#x.close()

#y = open('Attention_y_tokenizer.pickle', 'wb')
#pickle.dump(y_tokenizer, y)
#y.close()

#decode_sequence = open('Attention_decode_sequence.pickle', 'wb')
#pickle.dump(y_tokenizer, decode_sequence)
#decode_sequence.close()

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])


        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString = ''
    for i in input_seq:
        if((i != 0 and i != target_word_index['sostok']) and i != target_word_index['eostok']):
            newString = newString + reverse_target_word_index[i] + ' '
    return newString

def seq2text(input_seq):
    newString = ''
    for i in input_seq:
        if(i != 0):
            newString = newString + reverse_source_word_index[i]+' '
    return newString

In [ ]:
for i in range(0, 100):
    #print(DF['text'][i])
    print("TEXT : ", seq2text(x_tr[i]))
    print("ORIGINAL SUMMARY :", seq2summary(y_tr[i]))
    print("PREDICTED SUMMARY :", decode_sequence(x_tr[i].reshape(1, max_text_len)))
    print("\n")

**Evaluations :**

In [ ]:
DF.shape

In [ ]:
#liste des résumés de références
goldstandard = []
#liste des résumés générés
summ = []

#on en met 100 dans 2 listes
for i in range(0, 100000):
    goldstandard.append(seq2summary(y_tr[i]))

for i in range(0, 100000):
    summ.append(decode_sequence(x_tr[i].reshape(1, max_text_len)))
#on split les phrases pour créer une liste de liste de mots

GOLD = []
for i in goldstandard:
    GOLD.append(i.split())

SUMM = []
for i in summ:
    SUMM.append(i.split())

# on évalue la moyenne des scores
nltk.translate.bleu_score.corpus_bleu(GOLD, SUMM)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
goldstandard = []
summ = []

for i in range(0, 100000):
    goldstandard.append(seq2summary(y_tr[i]))

for i in range(0, 100000):
    summ.append(decode_sequence(x_tr[i].reshape(1, max_text_len)))

In [ ]:
GOLD = []
for i in goldstandard:
    GOLD.append(i.split())

SUMM = []
for i in summ:
    SUMM.append(i.split())

In [ ]:
nltk.translate.bleu_score.corpus_bleu(GOLD, SUMM)

In [ ]:
pip install rouge

In [ ]:
from rouge import Rouge 
rouge = Rouge()
for i, j in zip(goldstandard, summ):
    print("Original summary:",i)
    print("Predicted summary:",j)
    scores = rouge.get_scores(i, j)
    print(scores)
    print('\\n')

In [ ]:
hypothesis = "talk to your friends about your feelings talk to therapist "
reference = "problem solve ask for support prioritize self care take break engage in therapy help others"

In [ ]:
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)